In [ ]:
# Configurazione messaggi di logging
logging.basicConfig(
   level=logging.DEBUG,    # Garantisce che DEBUG passi
   format="%(asctime)s %(levelname)s %(name)s: %(message)s",
   datefmt="%H:%M:%S",
   force=True              # Per Jupyter/ambienti già configurati
)

# # Imposto il cluster Dask
# def main():
#    cluster = LocalCluster(
#       n_workers=4,           # Numero di processi indipendenti
#       threads_per_worker=1,  # Evita oversubscription della CPU
#       memory_limit='1GB',    # Limita RAM per worker
#       silence_logs=False
#    )
#    client = Client(cluster)
#    print(client.dashboard_link)

# if __name__ == "__main__":
#    main()

# if __name__ == "__main__":
#     print("OKKKKKK")
#     mp.set_start_method("fork")

#     from dask.distributed import Client, LocalCluster

#     cluster = LocalCluster(
#         n_workers=4,
#         threads_per_worker=1,
#         memory_limit="5GB"
#     )
#     client = Client(cluster)
#     print(client.dashboard_link)

# print("fine dei giochi")
# sys.exit()
# print("cazzo")

# Directories
## Home
common_dir_home = '/home/montanarini/ELNINO/'
dir_output_9 = 'output/Task-9_ROModel/'

## Datasets
common_dir_ds = '/nas/'
### pi-control
dir_pi = 'archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/piControl/'
dir_pi_tos = 'ocean/Omon/r1i1p1f1_r25/tos/*.nc'
### Bottini
dir_BOTTINI = 'BOTTINO/irods_data/stabilization-{}/r1i1p1f1/'
dir_bxxx = '{}/*.nc'

# Constants
secondi_mese = 60*60*24*30.43803025425 # Durata media mese: 365.256363051/12

# Keywords: datasets, variables and path completion
## Datsets
dataset_names = ['pi-control', 'b990','b025', 'b050', 'b065', 'b080', 'b100']
dataset_names_iteration = len(dataset_names)
## Completamento dei path dei bottini
bottini_paths = ['hist-1990', 'ssp585-2025', 'ssp585-2050', 'ssp585-2065', 'ssp585-2080', 'ssp585-2100']
## Definisco le liste di variaibli (Oceaniche)
var_names = ['tos', 'thetao']
var_names_iteration = len(var_names)

# Parametri
# valori da usare (i) confronto per violin-plot: 500, 360, 500; (ii) confronto per scatter-plot: [[1000, 360, 1000], [100, 360, 100]]
time_window = 500 # ultimi n anni del Dataset (non per hist e obs), default: 500
time_rm = 360 # Mesi in cui si applica la RM, default: 360
print("\nThe following set of parameters was selected:",
   "\n- Last n years of the bottino runs: ", time_window,
   "\n- Running Mean time, in months (for the historical detrending): ", time_rm)

# Flag per eseguire le cose una volta sola
flag_continental = True

# Dizionario dei path per i Datasets per ogni variabile
directories = {}
for i, datasetN in enumerate(dataset_names):
   directories[datasetN] = {}
   for var in var_names:
      # Aggiunta dei path non bottino
      if datasetN in ['pi-control']:
         directories[datasetN][var] = [common_dir_ds + dir_pi + 'ocean/Omon/r1i1p1f1_r25/' + var + '/*.nc']
      # Aggiunta dei path bottino
      elif datasetN in ['b990','b025', 'b050', 'b065', 'b080', 'b100']:
         directories[datasetN][var] = [(common_dir_ds + dir_BOTTINI).format(bottini_paths[i-1]) + 'Omon_r25/' + dir_bxxx.format(var_names[var_names.index(var)])]

# Liste per salvare i tempi impiegati per ogni dataset
time_begin_var = [None] * var_names_iteration
time_end_var = [None] * var_names_iteration
time_var = [None] * var_names_iteration
time_begin_ds = [None] * dataset_names_iteration
time_end_ds = [None] * dataset_names_iteration

# Iterazione sulle Variabili
for i, var in enumerate(var_names):
   
   time_begin_var[i] = datetime.now()

   # Seleziona solo la variaibile n ed escludi le altre
   # if var not in ['thetao']:
   #    time.sleep(1)
   #    time_end_var[i] = datetime.now()
   #    time_var[i] = (time_end_var[i]-time_begin_var[i])
   #    continue

   print(f"\n# Inizia l'elaborazione della variabile: {var}")

   # Iterazione sui Dataset
   for j, datasetN in enumerate(dataset_names):
      time_begin_ds[j] = datetime.now()
      
      # Display informazioni sulla directory 
      print(f"\n## Inizia l'elaborazione di: Dataset_{datasetN}\nCorresponding path: ", directories[datasetN][var][0])
      

      print("\nStart of Dataset opening/upload")

      ###################################################################################################################################
      #################################### (a) Discrimination based on the Dataset of origin ############################################

      if datasetN in ['b990','b025', 'b050', 'b065', 'b080', 'b100']:
         # Apertura dataset
         Dataset = xr.open_mfdataset(
            directories[datasetN][var][0],
            use_cftime=True,
            parallel=True,
            chunks={'time': 100, 'lev': -1, 'lat': 10, 'lon': 50} # NB: le dimensiioni non presenti (lev in tos ad esempio) vengono ignorate
         )
         
      elif datasetN in ['pi-control']:
         # Apertura dataset
         Dataset = xr.open_mfdataset(
            directories[datasetN][var][0],
            decode_times=True,
            use_cftime=True,
            parallel=True,
            chunks={'time': 100, 'lev': -1, 'lat': 10, 'lon': 50}
         )
                         
      # Riordinamento coordinate per evitare frammentazione dei chunks
      #Dataset = apf.sort_dataset_coords(Dataset)#.rechunk({'time': 10, 'lev': -1, 'lat': 10, 'lon': 50})
      
      # Discriminazione del range temporale:     
      max_year = Dataset['time'].max()
      min_year = Dataset['time'].min()
      ## Estremi per i Bottini
      sup_year_bott = cftime.DatetimeProlepticGregorian(int(max_year.dt.year), 1, 1)
      inf_year_bott = cftime.DatetimeProlepticGregorian(int(max_year.dt.year-time_window), 1, 1)
      date_format = '%Y-%m-%d'
      ## Limitazione del range temporale
      Dataset = Dataset.sel(time=slice(inf_year_bott, sup_year_bott))

      print("\nStart of time and space variable selection")
      time_begin_var[j] = datetime.now()

      if var in ['tos']:
         # Passaggio al DataArray
         campo = Dataset['tos'].sel(lat=slice(-5, 5), lon=slice(210, 270)) # Nino3 zone for SSTA: eastern equatorial Pacific SSTA (T_E)

         # Detrending del campo e calcolo dell'anomalia
         campo_detrend, anomaly_detrend, trend_hist, seasonal_cycle = apf.detrending(campo, time_rm)
         
         # Maschera continentale
         if flag_continental == True:
            continental_mask = np.isnan(campo)[0,...]
            flag_continental = False
         
         # Calcolo la derivata dell'anomalia
         anomaly_detrend = anomaly_detrend.chunk({'time': 100, 'lat': 10, 'lon': 50})  # Rechunking del campo dopo la funzione di detrending(). {'time': -1} se vuoi un unico blocco
         Danom_Dt = anomaly_detrend.differentiate('time') * secondi_mese

         # Condensazione ad array 1D, discriminazione sul tipo di variaibile: zonale (time, lat, lon) o verticale (time, lat, lon, lev)
         #campo_detrend = apf.global_mean(campo_detrend)
         anomaly_detrend = apf.global_mean(anomaly_detrend)
         Danom_Dt = apf.global_mean(Danom_Dt)

         # Salvataggio dei campi
         print("\nStart saving the evaluated variables. Dataset: ", datasetN, ", Variable :", var)
         ds = xr.Dataset(
            data_vars=dict(
               anomaly = anomaly_detrend,
               Danom_Dt = Danom_Dt,
            )
         ).to_netcdf(common_dir_home + dir_output_9 + f'Task-9_{datasetN}-{var}.nc', compute=True)

      elif var in ['thetao']:
         # Passaggio al DataArray
         campo = Dataset['thetao'].sel(lev=slice(0,300), lat=slice(-5, 5), lon=slice(140, 205)) # Warm Pool region for Thermocline Depth Anomaly (hw) (Original: [120, 205])), limitation to the Mixing Layer.
         
         # Controllo dei chunks
         ## Informazioni sui chunk
         chunks = campo.data.chunks  # tuple di tuple: una per dimensione
         dtype_size = campo.data.dtype.itemsize  # byte per elemento
         ## Calcola il numero di elementi per chunk (in genere prendiamo il primo per stimare)
         chunk_shape = tuple(dim[0] for dim in chunks)
         n_elements = np.prod(chunk_shape)
         bytes_per_chunk = n_elements * dtype_size
         print(f"Chunk shape: {chunk_shape}")
         print(f"Bytes per chunk: {bytes_per_chunk / 1e6:.2f} MB")

         # Detrending del campo e calcolo dell'anomalia
         campo_detrend, anomaly_detrend, trend_hist, seasonal_cycle = apf.detrending(campo, time_rm)
         
         # Preparazione al calcolo della termoclina
         ## Coordinate
         lev = campo_detrend['lev']
         lon = campo_detrend['lon']
         ## Definizione dimensioni estese per interpolazioni
         lev_ext = xr.DataArray(np.linspace(lev.min().values, lev.max().values, 500), dims=['lev'])
         lon_ext = np.linspace(lon.min(), lon.max(), 500)
         lon_ext_coord = xr.DataArray(lon_ext, dims=['lon_ext']) # Crea un DataArray per allineare le coordinat
         
         ## Media latitudinale
         campo_tofunc = apf.meridional_mean(campo_detrend)

         ## Applico l'interpolazione alla coordinata di profondità (z) -'lev'- per ogni punto di longitudine e ricalcolo il valore del campo nei nuovi punti
         # Ci sono più modi per interpolare:
         #  - Modo 1, interpolazione semplice: DataArrray.interp()
         #  - campo_tofunc_interp = campo_tofunc.interp(coords={'lev': lev_ext}, method="linear")  # Dask-friendly e vettorizzato.
         #  - Modo 2: interpolazione spline, più precisa e accurata. Si può ottenere solo con funzioni non Dask friendly di Scipy.interpolate: interp1d(), splrep() + splev(); CubicSpline()
         #  che sono da applicare via 'apply_ufunc()' o 'map_blocks()' più flessibile specialmente per datasets chunked. Sono riuscito a eseguire una spline solo con interp1d(), con le altre funzioni ho fallito.
         campo_tofunc_interp = apf.spline_interpolation_along_dim(
            campo_tofunc,
            lev_ext,
            "lev",
            "cubic"
         )#.transpose('time', 'lon', 'lev').chunk({'time': 100, 'lon': 50}) # Riordino le dimensioni in modo che ci siano per prima le non core ('time', 'lat') e poi le core ('lon', 'lev') e forzo la coerenza con un chunking
         
         # Calcolod della Termoclina 
         interpol = False # Flag per abilitare o meno l'interpolazione
         termocline, depth_max_grad, temp_max_grad = apf.thermocline(campo_tofunc_interp, interpol) # La termoclina restituitita ha valori positivi sull'asse 'lev' , quindi sono da invertire moltiplicando *(-1)
         
         ## Anomalia
         termocline_anom = termocline - termocline.mean('time')
         
         ## Calcolo della derivata della termoclina
         #termocline_anom = termocline_anom.chunk({'time': -1})  # Rechunking del campo dopo la funzione di detrending()
         Dtermocline_anom_Dt = termocline_anom.differentiate('time')
         
         ## Riduzione dimensionale ad array 1D
         termocline_anom = termocline_anom.mean('lon')*(-1)
         Dtermocline_anom_Dt = Dtermocline_anom_Dt.mean('lon')*secondi_mese*(-1)

         # Salvataggio dei campi
         print("\nStart saving the evaluated variables. Dataset: ", datasetN, ", Variable :", var)
            
         ds = xr.Dataset(
            data_vars=dict(
               termocline_anom = termocline_anom,
               Dtermocline_anom_Dt = Dtermocline_anom_Dt,
            )
         ).to_netcdf(common_dir_home + dir_output_9 + f'Task-9_{datasetN}-{var}.nc', compute=True)
      
      ### Uscita dal loop sui Dataset ###
   
      # Dataset time recap
      time_end_ds[j] = datetime.now()
      print("\nEnd dataset elaboration")
      print("Dataset {} took {}".format(datasetN, time_end_ds[j]-time_begin_ds[j]))

   ### Uscita dal loop sulle variabili ###
   
   # Variable time recap
   time_end_var[i] = datetime.now()
   time_var[i] = (time_end_var[i]-time_begin_var[i])
   print("\nEnd of variable evaluation")
   print("Variable {} took {}".format(var_names[i], time_var[i]))

# Time recap
print("\n## Timing recap")
for i, var in enumerate(var_names):
   print("\nOverall evaluation of ", var, " for all the Datasets took {}".format(time_var[i]))

print("\nOverall evaluation of the script was completed in {}".format(np.sum(time_var)))
print("\nEnd of the script")

# Chiusura client e cluster Dask
client.close()
cluster.close()

In [ ]:
"""
Task 9 Recharge Oscillator Model

- Script di calcolo delle _anomalie_ dei campi e le rispettive _derivate_ di 'tos' e 'thetao' per i dataset Bottino + Pi-control,
per poterle utilizzare come serie temporali nell'equazione del modello di Jin del Recharge-Oscillator attraverso la semplificazione del modello lineare di regressione.
Delle variaibili citate, dopo averle isolate nele regioni di definizione presenti nell'articolo di Jin 2021, viene eseguito il detrending, e la condensazione a serie temporali (in cui l'unica dimensione rimanenete è il tempo).
Per 'thetao' viene calcolata la termoclina attraverso 2 funzioni complesse Dask-friendly anche via chunks, la prima interpola sulla dimensione 'lev', la seconda calcola la termoclina.

Nota: Ho cercato di eseguire la miglior ottimizzazione possibile con Dask, attraverso anche la definizione dei 'chunks'.
Tuttavia, visto che lavoro solo su una piccola fetta del mondo (e non su tutto il mondo) i chunk non sono ottimizzati, sono troppo piccoli e mi viene dato un messaggio di WARNING:
"/home/montanarini/miniforge3/envs/ctl4b/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 30 times more chunks"
Ad ogni modo la pipeline di Dask costituisce la base di partenza per altri scripts.
"""
# import libraries
## System libraries
import sys
import logging
## General data analisys libraries 
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
## Time library
import cftime
from datetime import datetime
# Personal library
import AllePowerFunctions as apf

print(__name__, " has been launched on date:", datetime.now(), "\n")

# Configurazione messaggi di logging
logging.basicConfig(
   level=logging.DEBUG,    # Garantisce che DEBUG passi
   format="%(asctime)s %(levelname)s %(name)s: %(message)s",
   datefmt="%H:%M:%S",
   force=True              # Per Jupyter/ambienti già configurati
)

# Imposto il cluster Dask
def main():
   cluster = LocalCluster(
      n_workers=4,           # Numero di processi indipendenti
      threads_per_worker=1,  # Evita oversubscription della CPU
      memory_limit='5GB'    # Limita RAM per worker
   )
   client = Client(cluster)
   print(client.dashboard_link)

if __name__ == "__main__":
   main()

# Directories
## Home
common_dir_home = '/home/montanarini/ELNINO/'
dir_output_9 = 'output/Task-9_ROModel/'

## Datasets
common_dir_ds = '/nas/'
### pi-control
dir_pi = 'archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/piControl/'
dir_pi_tos = 'ocean/Omon/r1i1p1f1_r25/tos/*.nc'
### Bottini
dir_BOTTINI = 'BOTTINO/irods_data/stabilization-{}/r1i1p1f1/'
dir_bxxx = '{}/*.nc'

# Constants
secondi_mese = 60*60*24*30.43803025425 # Durata media mese: 365.256363051/12

# Keywords: datasets, variables and path completion
## Datsets
dataset_names = ['pi-control', 'b990','b025', 'b050', 'b065', 'b080', 'b100']
dataset_names_iteration = len(dataset_names)
## Completamento dei path dei bottini
bottini_paths = ['hist-1990','ssp585-2025', 'ssp585-2050', 'ssp585-2065', 'ssp585-2080', 'ssp585-2100']
## Definisco le liste di variaibli (Oceaniche)
var_names = ['tos', 'thetao']
var_names_iteration = len(var_names)

# Parametri
# valori da usare (i) confronto per violin-plot: 500, 360, 500; (ii) confronto per scatter-plot: [[1000, 360, 1000], [100, 360, 100]]
time_window = 500 # ultimi n anni del Dataset (non per hist e obs), default: 500
time_rm = 360 # Mesi in cui si applica la RM, default: 360
print("\nThe following set of parameters was selected:",
   "\n- Last n years of the bottino runs: ", time_window,
   "\n- Running Mean time, in months (for the historical detrending): ", time_rm)

# Flag per eseguire le cose una volta sola
flag_continental = True

# Dizionario dei path per i Datasets per ogni variabile
directories = {}
for i, datasetN in enumerate(dataset_names):
   directories[datasetN] = {}
   for var in var_names:
      # Aggiunta dei path non bottino
      if datasetN in ['pi-control']:
         directories[datasetN][var] = [common_dir_ds + dir_pi + 'ocean/Omon/r1i1p1f1_r25/' + var + '/*.nc']
      # Aggiunta dei path bottino
      elif datasetN in ['b990','b025', 'b050', 'b065', 'b080', 'b100']:
         directories[datasetN][var] = [(common_dir_ds + dir_BOTTINI).format(bottini_paths[i-1]) + 'Omon_r25/' + dir_bxxx.format(var_names[var_names.index(var)])]

# Liste per salvare i tempi impiegati per ogni dataset
time_begin_var = [None] * var_names_iteration
time_end_var = [None] * var_names_iteration
time_var = [None] * var_names_iteration
time_begin_ds = [None] * dataset_names_iteration
time_end_ds = [None] * dataset_names_iteration
      
# Iterazione sulle Variabili
for i, var in enumerate(var_names):
   
   time_begin_var[i] = datetime.now()

   # Seleziona solo la variaibile n ed escludi le altre
   # if var not in ['thetao']:
   #    time.sleep(1)
   #    time_end_var[i] = datetime.now()
   #    time_var[i] = (time_end_var[i]-time_begin_var[i])
   #    continue

   print(f"\n# Inizia l'elaborazione della variabile: {var}")

   # Iterazione sui Dataset
   for j, datasetN in enumerate(dataset_names):
      time_begin_ds[j] = datetime.now()
      
      # Display informazioni sulla directory 
      print(f"\n## Inizia l'elaborazione di: Dataset_{datasetN}\nCorresponding path: ", directories[datasetN][var][0])
      

      print("\nStart of Dataset opening/upload")

      ###################################################################################################################################
      #################################### (a) Discrimination based on the Dataset of origin ############################################

      if datasetN in ['b990','b025', 'b050', 'b065', 'b080', 'b100']:
         # Apertura dataset
         Dataset = xr.open_mfdataset(
            directories[datasetN][var][0],
            use_cftime=True,
            parallel=True,
            chunks={'time': 100, 'lev': -1, 'lat': 10, 'lon': 50} # NB: le dimensiioni non presenti (lev in tos ad esempio) vengono ignorate
         )
         
      elif datasetN in ['pi-control']:
         # Apertura dataset
         Dataset = xr.open_mfdataset(
            directories[datasetN][var][0],
            decode_times=True,
            use_cftime=True,
            parallel=True,
            chunks={'time': 100, 'lev': -1, 'lat': 10, 'lon': 50}
         )
                         
      # Riordinamento coordinate per evitare frammentazione dei chunks
      #Dataset = apf.sort_dataset_coords(Dataset)#.rechunk({'time': 10, 'lev': -1, 'lat': 10, 'lon': 50})
      
      # Discriminazione del range temporale:     
      max_year = Dataset['time'].max()
      min_year = Dataset['time'].min()
      ## Estremi per i Bottini
      sup_year_bott = cftime.DatetimeProlepticGregorian(int(max_year.dt.year), 1, 1)
      inf_year_bott = cftime.DatetimeProlepticGregorian(int(max_year.dt.year-time_window), 1, 1)
      date_format = '%Y-%m-%d'
      ## Limitazione del range temporale
      Dataset = Dataset.sel(time=slice(inf_year_bott, sup_year_bott))

      print("\nStart of time and space variable selection")
      time_begin_var[j] = datetime.now()

      if var in ['tos']:
         # Passaggio al DataArray
         campo = Dataset['tos'].sel(lat=slice(-5, 5), lon=slice(210, 270)) # Nino3 zone for SSTA: eastern equatorial Pacific SSTA (T_E)

         # Detrending del campo e calcolo dell'anomalia
         campo_detrend, anomaly_detrend, trend_hist, seasonal_cycle = apf.detrending(campo, time_rm)
         
         # Maschera continentale
         if flag_continental == True:
            continental_mask = np.isnan(campo)[0,...]
            flag_continental = False
         
         # Calcolo la derivata dell'anomalia
         anomaly_detrend = anomaly_detrend.chunk({'time': 100, 'lat': 10, 'lon': 50})  # Rechunking del campo dopo la funzione di detrending(). {'time': -1} se vuoi un unico blocco
         Danom_Dt = anomaly_detrend.differentiate('time') * secondi_mese

         # Condensazione ad array 1D, discriminazione sul tipo di variaibile: zonale (time, lat, lon) o verticale (time, lat, lon, lev)
         #campo_detrend = apf.global_mean(campo_detrend)
         anomaly_detrend = apf.global_mean(anomaly_detrend)
         Danom_Dt = apf.global_mean(Danom_Dt)

         # Salvataggio dei campi
         print("\nStart saving the evaluated variables. Dataset: ", datasetN, ", Variable :", var)
         ds = xr.Dataset(
            data_vars=dict(
               anomaly = anomaly_detrend,
               Danom_Dt = Danom_Dt,
            )
         ).to_netcdf(common_dir_home + dir_output_9 + f'Task-9_{datasetN}-{var}.nc', compute=True)

      elif var in ['thetao']:
         # Passaggio al DataArray
         campo = Dataset['thetao'].sel(lev=slice(0,300), lat=slice(-5, 5), lon=slice(140, 205)) # Warm Pool region for Thermocline Depth Anomaly (hw) (Original: [120, 205])), limitation to the Mixing Layer.
         
         # Controllo dei chunks
         ## Informazioni sui chunk
         chunks = campo.data.chunks  # tuple di tuple: una per dimensione
         dtype_size = campo.data.dtype.itemsize  # byte per elemento
         ## Calcola il numero di elementi per chunk (in genere prendiamo il primo per stimare)
         chunk_shape = tuple(dim[0] for dim in chunks)
         n_elements = np.prod(chunk_shape)
         bytes_per_chunk = n_elements * dtype_size
         print(f"Chunk shape: {chunk_shape}")
         print(f"Bytes per chunk: {bytes_per_chunk / 1e6:.2f} MB")

         # Detrending del campo e calcolo dell'anomalia
         campo_detrend, anomaly_detrend, trend_hist, seasonal_cycle = apf.detrending(campo, time_rm)
         
         # Preparazione al calcolo della termoclina
         ## Coordinate
         lev = campo_detrend['lev']
         lon = campo_detrend['lon']
         ## Definizione dimensioni estese per interpolazioni
         lev_ext = xr.DataArray(np.linspace(lev.min().values, lev.max().values, 500), dims=['lev'])
         lon_ext = np.linspace(lon.min(), lon.max(), 500)
         lon_ext_coord = xr.DataArray(lon_ext, dims=['lon_ext']) # Crea un DataArray per allineare le coordinat
         
         ## Media latitudinale
         campo_tofunc = apf.meridional_mean(campo_detrend)

         ## Applico l'interpolazione alla coordinata di profondità (z) -'lev'- per ogni punto di longitudine e ricalcolo il valore del campo nei nuovi punti
         # Ci sono più modi per interpolare:
         #  - Modo 1, interpolazione semplice: DataArrray.interp()
         #  - campo_tofunc_interp = campo_tofunc.interp(coords={'lev': lev_ext}, method="linear")  # Dask-friendly e vettorizzato.
         #  - Modo 2: interpolazione spline, più precisa e accurata. Si può ottenere solo con funzioni non Dask friendly di Scipy.interpolate: interp1d(), splrep() + splev(); CubicSpline()
         #  che sono da applicare via 'apply_ufunc()' o 'map_blocks()' più flessibile specialmente per datasets chunked. Sono riuscito a eseguire una spline solo con interp1d(), con le altre funzioni ho fallito.
         campo_tofunc_interp = apf.spline_interpolation_along_dim(
            campo_tofunc,
            lev_ext,
            "lev",
            "cubic"
         )#.transpose('time', 'lon', 'lev').chunk({'time': 100, 'lon': 50}) # Riordino le dimensioni in modo che ci siano per prima le non core ('time', 'lat') e poi le core ('lon', 'lev') e forzo la coerenza con un chunking
         
         # Calcolod della Termoclina 
         interpol = False # Flag per abilitare o meno l'interpolazione
         termocline, depth_max_grad, temp_max_grad = apf.thermocline(campo_tofunc_interp, interpol) # La termoclina restituitita ha valori positivi sull'asse 'lev' , quindi sono da invertire moltiplicando *(-1)
         
         ## Anomalia
         termocline_anom = termocline - termocline.mean('time')
         
         ## Calcolo della derivata della termoclina
         #termocline_anom = termocline_anom.chunk({'time': -1})  # Rechunking del campo dopo la funzione di detrending()
         Dtermocline_anom_Dt = termocline_anom.differentiate('time')
         
         ## Riduzione dimensionale ad array 1D
         termocline_anom = termocline_anom.mean('lon')*(-1)
         Dtermocline_anom_Dt = Dtermocline_anom_Dt.mean('lon')*secondi_mese*(-1)

         # Salvataggio dei campi
         print("\nStart saving the evaluated variables. Dataset: ", datasetN, ", Variable :", var)
            
         ds = xr.Dataset(
            data_vars=dict(
               termocline_anom = termocline_anom,
               Dtermocline_anom_Dt = Dtermocline_anom_Dt,
            )
         ).to_netcdf(common_dir_home + dir_output_9 + f'Task-9_{datasetN}-{var}.nc', compute=True)
      
      ### Uscita dal loop sui Dataset ###
   
      # Dataset time recap
      time_end_ds[j] = datetime.now()
      print("\nEnd dataset elaboration")
      print("Dataset {} took {}".format(datasetN, time_end_ds[j]-time_begin_ds[j]))

   ### Uscita dal loop sulle variabili ###
   
   # Variable time recap
   time_end_var[i] = datetime.now()
   time_var[i] = (time_end_var[i]-time_begin_var[i])
   print("\nEnd of variable evaluation")
   print("Variable {} took {}".format(var_names[i], time_var[i]))

# Time recap
print("\n## Timing recap")
for i, var in enumerate(var_names):
   print("\nOverall evaluation of ", var, " for all the Datasets took {}".format(time_var[i]))

print("\nOverall evaluation of the script was completed in {}".format(np.sum(time_var)))
print("\nEnd of the script")

# Chiusura client e cluster Dask
client.close()
cluster.close()

print("\nEnd of the script")
sys.exit()

In [ ]:
# Funzione Termoclina funzionante
def thermocline(
    campo: xr.DataArray,
    lon_ext: np.ndarray,
    chunks: Optional[dict] = None,
    levels: Optional[np.ndarray] = None,
    latitude: Optional[np.ndarray] = None,
    longitude: Optional[np.ndarray] = None
) -> Tuple[xr.DataArray, xr.DataArray, xr.DataArray]:
    """
    Calcola la termoclina come punto di massimo gradiente verticale.
    Versione sia per Xarray, Dask-friendly anche per Dataset chunakti, che per Numpy.ndarray.
    
    Parameters
    ----------
    campo :
        1° caso, xr.DataArray: campo di temperatura con dimensioni (time, lev, lat, lon) o subset
        2° caso, numpy.ndarray: è importante che le dimensioni siano ordinate come mostrato.
            Se le dimensioni sono 3 vanno portate a 2, altrimenti è ambiguo. NB in questo caso il Tempo viene ridotto, mediato via,
            mentre la versione Xarray parallelizza su 'time'. Adattala nel caso tale dimensione serva.
            Questo ramo NON è dask friendly, lo può diventare via 'apply_ufunc' che puoi eseguire dentro la funzione stessa (questa, modificandola) che fuori, cioè nel codice.
    lon_ext : np.ndarray
        Nuove longitudini su cui interpolare la termoclina
    levels : np.ndarray, optional
        Livelli di profondità (solo per input numpy)
    latitude : np.ndarray, optional  
        Latitudini (solo per input numpy)
    longitude : np.ndarray, optional
        Longitudini (solo per input numpy)
    
    Returns
    -------
    termoclina_interp : xr.DataArray
        Profondità della termoclina interpolata su lon_ext
    depth_max_grad : xr.DataArray  
        Profondità della termoclina per ogni longitudine originale
    temp_max_grad : xr.DataArray
        Temperatura alla termoclina per ogni longitudine originale
    """
    if isinstance(campo, xr.DataArray):
        # 0. Preparazione
        ## 0.1 Verifica Chunks
        if isinstance(campo.data, dask_array.Array):
            if chunks:
                campo = campo.chunk(chunks)
                # lev non deve essere chunkata
                lev_chunks = chunks[campo.dims.index('lev')] if 'lev' in campo.dims else ()
                if len(lev_chunks) > 1 or (lev_chunks and lev_chunks[0] != campo.lev.size):
                    raise ValueError(
                        "La dimensione 'lev' deve essere interamente contenuta in un singolo chunk. "
                        "Usa campo.chunk({'lev': -1}) prima di chiamare questa funzione."
                    )                
            else:
                # Chunks di default
                campo = campo.chunk({'time': 120, 'lev': -1, 'lon': 27})            
        
        ## 0.2 Clona il DataArray per non modificare l'originale
        campo_work = campo.copy()
        
        ## 0.3 Media su 'lat'
        if 'lat' in campo_work.dims:
            try:
                campo_work = meridional_mean(campo_work, latitude=None, mask=None, skip_nan=True, skip_inf=True)
            except:
                # Fallback a media semplice se meridional_mean non disponibile
                campo_work = campo_work.mean('lat')
                logger.debug("Fallback, media latitudinale non pesata")

        ## 0.4 Gradiente
        dT_dlev = campo_work.differentiate('lev')
        abs_dT_dlev = np.abs(dT_dlev)
        
        ## 0.5 Massimo del gradiente usando funzione apposita
        #il fancy indexing con 'isel' non è gestito da Dask, quindi non posso fare semplicemente come segue,
        #devo passare da numpy e di conseguenza apply_ufunc
        #idx_max_grad = abs_dT_dlev.argmax(dim='lev')
        #depth_max_grad = campo_work['lev'].isel(lev=idx_max_grad)
        #temp_max_grad = campo_work.isel(lev=idx_max_grad)
        def _find_max_grad(temperature, levels, abs_grad):
            """
            Trova indice, profondità e temperatura del massimo gradiente.
            """
            # Trova indice del massimo gradiente
            idx_max = np.nanargmax(abs_grad, axis=0)
            
            # Estrai profondità corrispondente
            depth_max = np.take_along_axis(
                levels.reshape(-1, 1), 
                idx_max.reshape(1, -1), 
                axis=0
            ).squeeze()
            
            # Estrai temperatura corrispondente
            temp_max = np.take_along_axis(
                temperature, 
                idx_max.reshape(1, -1), 
                axis=0
            ).squeeze()
            
            return depth_max, temp_max, idx_max
        
        ### 0.5.1 Applica la funzione usando 'apply_ufunc'
        depth_max_grad, temp_max_grad, idx_max_grad = xr.apply_ufunc(
            _find_max_grad,                                                     # Funzione da applicare
            campo_work,                                                         # Input -1° parametro-: Campo a cui applicarla
            campo_work['lev'],                                                  # Input -2° parametro-: Vettore delle coordinate dei livelli
            abs_dT_dlev,                                                        # Input -3° parametro-: Valore assoluto gradiente
            input_core_dims=[['lev', 'lon'], ['lev'], ['lev', 'lon']],          # Dimensioni da trattare come "core"
            output_core_dims=[['lon'], ['lon'], ['lon']],                       # Dimensioni degli output
            vectorize=True,                                                     # Vectorizza su tutte le altre dimensioni (es: time)
            dask='parallelized',                                                # Forza la parallelizzazione
            output_dtypes=[campo_work['lev'].dtype, campo_work.dtype, np.int64] # Data type of the retourned array
        )
        # 1. Interpolazione
        ## 1.1 funzione interpolante per array 1D su lon_ext
        def _spline_cubic_1d(x_old, y_old, x_new):
              """Funzione interna per interpolazione spline 1D"""
              # Gestione valori NaN
              valid_mask = np.isfinite(y_old)
              if np.sum(valid_mask) < 2:
                 return np.full_like(x_new, np.nan)
              
              x_valid = x_old[valid_mask]
              y_valid = y_old[valid_mask]
               
              try:
                 # Interpolazione spline cubica
                 interpolator = interp1d(
                    x_valid, y_valid, 
                    kind='cubic', 
                    bounds_error=False,
                    fill_value=np.nan,
                    assume_sorted=True
                 )
                 return interpolator(x_new)
              except (ValueError, TypeError) as e:
                 # Fallback a interpolazione lineare
                 logger.debug(f"Spline failed, using linear: {e}")
                 return np.interp(x_new, x_valid, y_valid, left=np.nan, right=np.nan)

        ## 1.2 Applica interpolazione usando 'apply_ufunc'
        termoclina_interp = xr.apply_ufunc(
              _spline_cubic_1d,                                               # Funzione da applicare
              campo_work['lon'],                                              # Input: Vettore delle coordinate ascisse originale: x_old
              depth_max_grad,                                                 # Input: Vettore delle coordinate ordinate originale: y_old
              lon_ext,                                                        # Input: Vettore delle coordinate x di output (per la spline), nuova ascissa: x_new
              input_core_dims=[['lon'], ['lon'], ['lon_ext']],                # Dimensioni da trattare come "core" (1D)
              output_core_dims=[['lon_ext']],                                 # Dimensione degli output
              vectorize=True,                                                 # Vectorizza su tutte le altre dimensioni (es: time)
              dask='parallelized',                                            # Forza la parallelizzazione
              output_dtypes=[depth_max_grad.dtype],                           # Data type of the retourned array
              dask_gufunc_kwargs={'output_sizes': {'lon_ext': len(lon_ext)}}
        )

        ## 1.3 Aggiungi coordinate al risultato
        termoclina_interp = termoclina_interp.assign_coords({'lon_ext': lon_ext})
        termoclina_interp = termoclina_interp.rename({'lon_ext': 'lon'})
        
        # 2. Controllo (opzionale - da eseguire solo se non lazy)
        if not isinstance(termoclina_interp.data, dask_array.Array):
            for output in [termoclina_interp, depth_max_grad, temp_max_grad]:
                if np.any(np.isnan(output)):
                    logger.warning("Risultato contiene NaN")
        
        return termoclina_interp, depth_max_grad, temp_max_grad
    
    elif isinstance(campo, xr.Dataset):
        raise ValueError("Passa un DataArray invece di un Dataset")
    
    elif isinstance(campo, np.ndarray):
        # 0. Preparazione
        ## 0.1 Verifica se le coordinate sono state fornite
        if longitude is None:
            raise ValueError("Se 'campo' è un numpy.ndarray, l'array delle coordinate 'lon' deve essere fornito.")
        
        ## 0.2 Media su tempo e latitudine
        if np.ndim(campo) == 4:
            # Media sull'asse 0 ('time')
            campo = np.average(campo, axis=0) 
            # Media sull'asse 1 (ora asse 0) ('lat')
            campo = meridional_mean(campo, latitude=latitude, mask=None, skip_nan=True, skip_inf=True)
        elif (np.ndim(campo) == 3) or (np.ndim(campo)>4):
            print("Ambiguità con 3 dimensioni, riducile a 2!")
        
        # 1. Massimo del gradiente (termoclina)
        # 'campo' deve essereun array 2D (lon, lev)
        # 1.0 Calcola il gradiente verticale (derivata rispetto al livello)
        DT_Dlev = np.gradient(campo, levels, axis=0) #L'argomento 'levels' specifica la spaziatura (i livelli stessi) per un gradiente non uniforme.
        abs_DT_Dlev = np.abs(DT_Dlev)
        # 1.1 Trova l'indice del massimo gradiente lungo l'asse 'levels' con argmax
        idx_max_grad = np.argmax(abs_DT_Dlev, axis=0) # Risultato: un array 1D (lunghezza 'lon')
        # 1.2 Trova la profondita'/livello del massimo gradiente. np.take(a, indices, axis) estrae l'elemento a quell'indice lungo l'asse.
        # Vogliamo estrarre il valore di 'lev' [metri] che corrisponde all'indice di 'lon' trovato per ogni 'lon'.
        depth_max_grad = levels[idx_max_grad]
        # 1.3 Livelli di profondità su cui si registra la termoclina per ogni lon
        lon_indices = np.arange(len(longitude))
        # 1.4 Usa gli indici (lon, lev) per estrarre i valori dal campo (le temperature)
        temp_max_grad = campo[idx_max_grad, lon_indices]
        
        # 2. Interpolazione, Cubica: usiamo lon come X e depth_max_grad come Y e lo valutiamo (interpolando) sulla longitudine estesa
        termoclina_interp = CubicSpline(longitude, depth_max_grad, bc_type='natural')(lon_ext)
        for output in [termoclina_interp, depth_max_grad, temp_max_grad]:
            if np.any(np.isnan(output)):
                logger.warning("Risultato contiene NaN")
        return termoclina_interp, depth_max_grad, temp_max_grad

    else:
        raise TypeError("Tipo di input non supportato. Usa xr.DataArray o np.ndarray.")

In [ ]:
# BACKUP
# Funzione termoclina OLD non allineata a versione per applicazione per chunks, versione NON dask-friendly.

import numpy as np
import xarray as xr
import dask.array as dask_array
from scipy.interpolate import interp1d
import logging
from typing import Tuple, Optional, Union
import AllePowerFunctions as apf

logger = logging.getLogger(__name__)

def thermocline(
    campo: xr.DataArray,
    lon_ext: np.ndarray,
    levels: Optional[np.ndarray] = None,
    latitude: Optional[np.ndarray] = None,
    longitude: Optional[np.ndarray] = None
) -> Tuple[xr.DataArray, xr.DataArray, xr.DataArray]:
    """
    Calcola la termoclina come punto di massimo gradiente verticale.
    Versione Dask-friendly.
    
    Parameters
    ----------
    campo :
        1° caso, xr.DataArray: campo di temperatura con dimensioni (time, lev, lat, lon) o subset
        2° caso, numpy.ndarray: è importante che le dimensioni siano ordinate come mostrato. Se le dimensioni sono 3 vanno portate a 2, altrimenti è ambiguo.
    lon_ext : np.ndarray
        Nuove longitudini su cui interpolare la termoclina
    levels : np.ndarray, optional
        Livelli di profondità (solo per input numpy)
    latitude : np.ndarray, optional  
        Latitudini (solo per input numpy)
    longitude : np.ndarray, optional
        Longitudini (solo per input numpy)
    
    Returns
    -------
    termoclina_interp : xr.DataArray
        Profondità della termoclina interpolata su lon_ext
    depth_max_grad : xr.DataArray  
        Profondità della termoclina per ogni longitudine originale
    temp_max_grad : xr.DataArray
        Temperatura alla termoclina per ogni longitudine originale
    """
    if isinstance(campo, xr.DataArray):
        # 0. Preparazione
        ## 0.1 Verifica Chunks
        if isinstance(campo.data, dask_array.Array):
            lev_chunks = campo.chunks[campo.dims.index('lev')] if 'lev' in campo.dims else ()
            if len(lev_chunks) > 1 or (lev_chunks and lev_chunks[0] != campo.lev.size):
                raise ValueError(
                    "La dimensione 'lev' deve essere interamente contenuta in un singolo chunk. "
                    "Usa campo.chunk({'lev': -1}) prima di chiamare questa funzione."
                )
        
        ## 0.2 Clona il DataArray per non modificare l'originale
        campo_work = campo.copy()
        
        ## 0.3 Media su 'time' e 'lat'
        if 'time' in campo_work.dims:
            campo_work = campo_work.mean('time')
        if 'lat' in campo_work.dims:
            try:
                campo_work = apf.meridional_mean(campo_work, latitude=None, mask=None, skip_nan=True, skip_inf=True)
            except:
                # Fallback a media semplice se meridional_mean non disponibile
                campo_work = campo_work.mean('lat')
                logger.debug("Fallback, media latitudinale non pesata")
        
        ## 0.4 Gradiente
        dT_dlev = campo_work.differentiate('lev')
        abs_dT_dlev = np.abs(dT_dlev)
        ## 0.5  Massimo del gradiente (operazione non Dask se il Dataset è chunked)
        idx_max_grad = abs_dT_dlev.argmax(dim='lev')
        depth_max_grad = campo_work['lev'].isel(lev=idx_max_grad)
        temp_max_grad = campo_work.isel(lev=idx_max_grad)
        
        # 1. Interpolazione
        ## 1.1 funzione interpolante per array 1D su lon_ext
        def _spline_cubic_1d(x_old, y_old, x_new):
            """Funzione interna per interpolazione spline 1D"""
            # Gestione valori NaN
            valid_mask = np.isfinite(y_old)
            if np.sum(valid_mask) < 2:
                return np.full_like(x_new, np.nan)
            
            x_valid = x_old[valid_mask]
            y_valid = y_old[valid_mask]
            
            try:
                # Interpolazione spline cubica
                interpolator = interp1d(
                    x_valid, y_valid, 
                    kind='cubic', 
                    bounds_error=False,
                    fill_value=np.nan,
                    assume_sorted=True
                )
                return interpolator(x_new)
            except (ValueError, TypeError) as e:
                # Fallback a interpolazione lineare
                logger.debug(f"Spline failed, using linear: {e}")
                return np.interp(x_new, x_valid, y_valid, left=np.nan, right=np.nan)
        
        # 1.2 Applica interpolazione usando 'apply_ufunc'
        termoclina_interp = xr.apply_ufunc(
            _spline_cubic_1d,                                               # funzione da applicare
            campo_work['lon'],                                              # ascissa originale: x_old
            depth_max_grad,                                                 # dati: y_old
            lon_ext,                                                        # nuova ascissa: x_new
            input_core_dims=[['lon'], ['lon'], ['lon_ext']],                # specifica le dimensioni da trattare come "core" (1D)
            output_core_dims=[['lon_ext']],                                 # la dimensione di output
            vectorize=True,                                                 # vectorizza su tutte le altre dimensioni (es: lon)
            dask='parallelized',                                            # Dask solo per funzioni dask-compatibili
            output_dtypes=[depth_max_grad.dtype],                           # data type of the retourned array
            dask_gufunc_kwargs={'output_sizes': {'lon_ext': len(lon_ext)}}
        )
        
        # 2. Aggiustamenti e Checks
        ## 2.1 Aggiungi coordinate al risultato
        termoclina_interp = termoclina_interp.assign_coords({'lon_ext': lon_ext})
        termoclina_interp = termoclina_interp.rename({'lon_ext': 'lon'})
        
        ## 2.2 Controllo (opzionale - da eseguire solo se non lazy)
        if not isinstance(termoclina_interp.data, dask_array.Array):
            for output in [termoclina_interp, depth_max_grad, temp_max_grad]:
                if np.any(np.isnan(output)):
                    logger.warning("Risultato contiene NaN")
        
        return termoclina_interp, depth_max_grad, temp_max_grad
    
    elif isinstance(campo, xr.Dataset):
        raise ValueError("Passa un DataArray invece di un Dataset")
    
    elif isinstance(campo, np.ndarray):
        # 0. Preparazione
        ## 0.1 Verifica se le coordinate sono state fornite
        if longitude is None:
            raise ValueError("Se 'campo' è un numpy.ndarray, l'array delle coordinate 'lon' deve essere fornito.")
        
        ## 0.2 Media su tempo e latitudine
        if np.ndim(campo) == 4:
            # Media sull'asse 0 ('time')
            campo = np.average(campo, axis=0) 
            # Media sull'asse 1 (ora asse 0) ('lat')
            campo = meridional_mean(campo, latitude=latitude, mask=None, skip_nan=True, skip_inf=True)
        elif (np.ndim(campo) == 3) or (np.ndim(campo)>4):
            print("Ambiguità con 3 dimensioni, riducile a 2!")
        
        # 1. Massimo del gradiente (termoclina)
        # 'campo' deve essereun array 2D (lon, lev)
        # 1.0 Calcola il gradiente verticale (derivata rispetto al livello)
        DT_Dlev = np.gradient(campo, levels, axis=0) #L'argomento 'levels' specifica la spaziatura (i livelli stessi) per un gradiente non uniforme.
        abs_DT_Dlev = np.abs(DT_Dlev)
        # 1.1 Trova l'indice del massimo gradiente lungo l'asse 'levels' con argmax
        idx_max_grad = np.argmax(abs_DT_Dlev, axis=0) # Risultato: un array 1D (lunghezza 'lon')
        # 1.2 Trova la profondita'/livello del massimo gradiente. np.take(a, indices, axis) estrae l'elemento a quell'indice lungo l'asse.
        # Vogliamo estrarre il valore di 'lev' [metri] che corrisponde all'indice di 'lon' trovato per ogni 'lon'.
        depth_max_grad = levels[idx_max_grad]
        # 1.3 Livelli di profondità su cui si registra la termoclina per ogni lon
        lon_indices = np.arange(len(longitude))
        # 1.4 Usa gli indici (lon, lev) per estrarre i valori dal campo (le temperature)
        temp_max_grad = campo[idx_max_grad, lon_indices]
        
        # 2. Interpolazione, Cubica: usiamo lon come X e depth_max_grad come Y e lo valutiamo (interpolando) sulla longitudine estesa
        termoclina_interp = CubicSpline(longitude, depth_max_grad, bc_type='natural')(lon_ext)
        for output in [termoclina_interp, depth_max_grad, temp_max_grad]:
            if np.any(np.isnan(output)):
                logger.warning("Risultato contiene NaN")
        return termoclina_interp, depth_max_grad, temp_max_grad

    else:
        raise TypeError("Tipo di input non supportato. Usa xr.DataArray o np.ndarray.")

In [ ]:
# BACKUP
# Funzione termoclina per applicazione per chunks, versione dask-friendly, DS

import numpy as np
import xarray as xr
import dask.array as dask_array
from scipy.interpolate import interp1d
import logging
from typing import Tuple, Optional
import AllePowerFunctions as apf

logger = logging.getLogger(__name__)

def thermocline_dask_chunk(
    campo: xr.DataArray,
    lon_ext: np.ndarray,
    chunks: Optional[dict] = None
) -> Tuple[xr.DataArray, xr.DataArray, xr.DataArray]:
    """
    Calcola la termoclina come punto di massimo gradiente verticale
    Versione Dask-friendly per applicazione per chunks via 'apply_ufunc()' iterando/parallelizzando sulla dimensione tempo
    
    Parameters
    ----------
    campo :
        xr.DataArray: campo di temperatura con dimensioni (time, lev, lat, lon) o subset
    lon_ext : np.ndarray
        Nuove longitudini su cui interpolare la termoclina
    
    Returns
    -------
    termoclina_interp : xr.DataArray
        Profondità della termoclina interpolata su lon_ext
    depth_max_grad : xr.DataArray  
        Profondità della termoclina per ogni longitudine originale
    temp_max_grad : xr.DataArray
        Temperatura alla termoclina per ogni longitudine originale
    """
    if isinstance(campo, xr.DataArray):
        # 0. Preparazione
        ## 0.1 Verifica Chunks
        if isinstance(campo.data, dask_array.Array):
            if chunks:
                campo = campo.chunk(chunks)
                # lev non deve essere chunkata
                lev_chunks = chunks[campo.dims.index('lev')] if 'lev' in campo.dims else ()
                if len(lev_chunks) > 1 or (lev_chunks and lev_chunks[0] != campo.lev.size):
                    raise ValueError(
                        "La dimensione 'lev' deve essere interamente contenuta in un singolo chunk. "
                        "Usa campo.chunk({'lev': -1}) prima di chiamare questa funzione."
                    )                
            else:
                # Chunks di default
                campo = campo.chunk({'time': 120, 'lev': -1, 'lon': 27})            
        
        ## 0.2 Clona il DataArray per non modificare l'originale
        campo_work = campo.copy()
        
        ## 0.3 Media su 'lat'
        if 'lat' in campo_work.dims:
            try:
                campo_work = apf.meridional_mean(campo_work, latitude=None, mask=None, skip_nan=True, skip_inf=True)
            except:
                # Fallback a media semplice se meridional_mean non disponibile
                campo_work = campo_work.mean('lat')
                logger.debug("Fallback, media latitudinale non pesata")

        ## 0.4 Gradiente
        dT_dlev = campo_work.differentiate('lev')
        abs_dT_dlev = np.abs(dT_dlev)
        
        ## 0.5 Massimo del gradiente usando funzione apposita
        #il fancy indexing con 'isel' non è gestito da Dask, quindi non posso fare semplicemente come segue,
        #devo passare da numpy e di conseguenza apply_ufunc
        #idx_max_grad = abs_dT_dlev.argmax(dim='lev')
        #depth_max_grad = campo_work['lev'].isel(lev=idx_max_grad)
        #temp_max_grad = campo_work.isel(lev=idx_max_grad)
        def _find_max_grad(temperature, levels, abs_grad):
            """
            Trova indice, profondità e temperatura del massimo gradiente.
            """
            # Trova indice del massimo gradiente
            idx_max = np.nanargmax(abs_grad, axis=0)
            
            # Estrai profondità corrispondente
            depth_max = np.take_along_axis(
                levels.reshape(-1, 1), 
                idx_max.reshape(1, -1), 
                axis=0
            ).squeeze()
            
            # Estrai temperatura corrispondente
            temp_max = np.take_along_axis(
                temperature, 
                idx_max.reshape(1, -1), 
                axis=0
            ).squeeze()
            
            return depth_max, temp_max, idx_max
        
        ### 0.5.1 Applica la funzione usando 'apply_ufunc'
        depth_max_grad, temp_max_grad, idx_max_grad = xr.apply_ufunc(
            _find_max_grad,
            campo_work,
            campo_work['lev'],
            abs_dT_dlev,
            input_core_dims=[['lev', 'lon'], ['lev'], ['lev', 'lon']],
            output_core_dims=[['lon'], ['lon'], ['lon']],
            vectorize=True,
            dask='parallelized',
            output_dtypes=[campo_work['lev'].dtype, campo_work.dtype, np.int64]
        )
        
        # 1. Interpolazione
        ## 1.1 funzione interpolante per array 1D su lon_ext
        def _spline_cubic_1d(x_old, y_old, x_new):
            """Funzione interna per interpolazione spline 1D"""
            # Gestione valori NaN
            valid_mask = np.isfinite(y_old)
            if np.sum(valid_mask) < 2:
                return np.full_like(x_new, np.nan)
            
            x_valid = x_old[valid_mask]
            y_valid = y_old[valid_mask]
            
            try:
                # Interpolazione spline cubica
                interpolator = interp1d(
                    x_valid, y_valid, 
                    kind='cubic', 
                    bounds_error=False,
                    fill_value=np.nan,
                    assume_sorted=True
                )
                return interpolator(x_new)
            except (ValueError, TypeError) as e:
                # Fallback a interpolazione lineare
                logger.debug(f"Spline failed, using linear: {e}")
                return np.interp(x_new, x_valid, y_valid, left=np.nan, right=np.nan)
        
        ## 1.2 Applica interpolazione usando 'apply_ufunc'
        termoclina_interp = xr.apply_ufunc(
            _spline_cubic_1d,                                               # funzione da applicare
            campo_work['lon'],                                              # ascissa originale: x_old
            depth_max_grad,                                                 # dati: y_old
            lon_ext,                                                        # nuova ascissa: x_new
            input_core_dims=[['lon'], ['lon'], ['lon_ext']],                # specifica le dimensioni da trattare come "core" (1D)
            output_core_dims=[['lon_ext']],                                 # la dimensione di output
            vectorize=True,                                                 # vectorizza su tutte le altre dimensioni (es: lon)
            dask='parallelized',                                            # Dask solo per funzioni dask-compatibili
            output_dtypes=[depth_max_grad.dtype],                           # data type of the retourned array
            dask_gufunc_kwargs={'output_sizes': {'lon_ext': len(lon_ext)}}
        )
        
        # 2. Aggiustamenti e Checks
        ## 2.1 Aggiungi coordinate al risultato
        termoclina_interp = termoclina_interp.assign_coords({'lon_ext': lon_ext})
        termoclina_interp = termoclina_interp.rename({'lon_ext': 'lon'})
        
        ## 2.2 Controllo (opzionale - da eseguire solo se non lazy)
        if not isinstance(termoclina_interp.data, dask_array.Array):
            for output in [termoclina_interp, depth_max_grad, temp_max_grad]:
                if np.any(np.isnan(output)):
                    logger.warning("Risultato contiene NaN")
        
        return termoclina_interp, depth_max_grad, temp_max_grad
    else:
        raise TypeError("Tipo di input non supportato. Usa xr.DataArray o np.ndarray.")

In [ ]:
----------------------------------------------------------------
# Funzione complessa non testata di DS che dovrebbe evitare la fancy indexing, ma ho già risolto questo problema. non la userei/considererei ulteriormente. da eliminare al più presto in caso 
def thermocline_dask_large_datasets(
    campo: xr.DataArray,
    lon_ext: np.ndarray,
    chunks: Optional[dict] = None
) -> Tuple[xr.DataArray, xr.DataArray, xr.DataArray]:
    """
    Versione ottimizzata per dataset LARGE con chunk temporali.
    """
    # 1. ASSICURA I CHUNKS CORRETTI
    if chunks:
        campo = campo.chunk(chunks)
    else:
        # Chunks di default per il tuo caso
        campo = campo.chunk({'time': 120, 'lev': -1, 'lon': 27})
    
    # 2. MEDIA SU LAT se presente (lazy)
    if 'lat' in campo.dims:
        campo = campo.mean('lat')
    
    # 3. FUNZIONE per processare CHUNK TEMPORALI
    def _process_time_chunk(temp_chunk, lev, lon):
        """
        Processa un chunk temporale (time, lev, lon) -> (time, lon)
        """
        n_time, n_lev, n_lon = temp_chunk.shape
        depth_results = np.empty((n_time, n_lon), dtype=temp_chunk.dtype)
        temp_results = np.empty((n_time, n_lon), dtype=temp_chunk.dtype)
        
        for t in range(n_time):
            for i in range(n_lon):
                profile = temp_chunk[t, :, i]
                
                # Gradiente (differenza finita)
                grad = np.abs(np.gradient(profile, lev))
                
                if np.all(np.isnan(grad)):
                    depth_results[t, i] = np.nan
                    temp_results[t, i] = np.nan
                else:
                    idx_max = np.nanargmax(grad)
                    depth_results[t, i] = lev[idx_max]
                    temp_results[t, i] = profile[idx_max]
        
        return depth_results, temp_results
    
    # 4. APPLICA SU OGNI CHUNK TEMPORALE (lazy!)
    lev_vals = campo['lev'].values
    lon_vals = campo['lon'].values
    
    if isinstance(campo.data, dask_array.Array):
        # Usa map_blocks per processare chunk temporali
        depth_temp_data = dask_array.map_blocks(
            lambda x: _process_time_chunk(x, lev_vals, lon_vals),
            campo.data,
            dtype=campo.dtype,
            chunks=(campo.chunks[0], campo.chunks[2]),  # (time_chunks, lon)
            new_axis=None
        )
        
        # depth_temp_data è una tupla
        depth_max_grad = xr.DataArray(
            depth_temp_data[0],
            dims=['time', 'lon'],
            coords={'time': campo.time, 'lon': campo.lon}
        )
        temp_max_grad = xr.DataArray(
            depth_temp_data[1],
            dims=['time', 'lon'],
            coords={'time': campo.time, 'lon': campo.lon}
        )
    else:
        # Versione numpy (non dovresti usarla per dataset così grandi)
        depth_arr, temp_arr = _process_time_chunk(campo.data, lev_vals, lon_vals)
        depth_max_grad = xr.DataArray(depth_arr, dims=['time', 'lon'], 
                                     coords={'time': campo.time, 'lon': campo.lon})
        temp_max_grad = xr.DataArray(temp_arr, dims=['time', 'lon'],
                                    coords={'time': campo.time, 'lon': campo.lon})
    
    # 5. INTERPOLAZIONE PER OGNI TIME STEP (lazy!)
    def _interpolate_per_time(x_old, y_old, x_new):
        """Interpolazione per ogni time step separatamente."""
        n_time = y_old.shape[0] if y_old.ndim > 1 else 1
        n_new = len(x_new)
        
        if y_old.ndim == 1:
            y_old = y_old.reshape(1, -1)
        
        result = np.empty((n_time, n_new), dtype=y_old.dtype)
        
        for t in range(n_time):
            valid_mask = np.isfinite(y_old[t, :])
            if np.sum(valid_mask) < 2:
                result[t, :] = np.nan
            else:
                result[t, :] = np.interp(
                    x_new, 
                    x_old[valid_mask], 
                    y_old[t, valid_mask],
                    left=np.nan, right=np.nan
                )
        
        return result.squeeze()
    
    # Applica interpolazione mantenendo chunks temporali
    termoclina_interp = xr.apply_ufunc(
        _interpolate_per_time,
        depth_max_grad['lon'],
        depth_max_grad,
        lon_ext,
        input_core_dims=[['lon'], ['time', 'lon'], ['lon_ext']],
        output_core_dims=[['time', 'lon_ext']],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[depth_max_grad.dtype],
        dask_gufunc_kwargs={'output_sizes': {'lon_ext': len(lon_ext)}}
    )
    
    # Aggiungi coordinate
    termoclina_interp = termoclina_interp.assign_coords({
        'time': campo.time,
        'lon_ext': lon_ext
    }).rename({'lon_ext': 'lon'})
    
    return termoclina_interp, depth_max_grad, temp_max_grad